# 4 Spark SQL and Dataframes: Introduction to Built-in Data Sources



In [ ]:
from pyspark.sql import SparkSession

spark: SparkSession = SparkSession.builder.appName("SparkSQLExampleApp").getOrCreate()
spark

In [ ]:
from pyspark.sql import DataFrame


data_path_csv = "data/flights/departuredelays.csv"

df: DataFrame = spark.read.option("SampleRatio", 0.1).csv(data_path_csv, header=True, inferSchema=True)
df.count(), df.columns, df.show(5), df.printSchema()

In [ ]:
df.createOrReplaceTempView("temp_table")

In [ ]:
spark.sql("select * from temp_table limit 10").show()

In [ ]:
# spark.sql("select * from temp_table where distance > 1000 order by distance desc").show()
spark.sql("select * from temp_table where delay > 120 and origin = 'SFO' and destination = 'ORD'").show(5)

In [ ]:
from pyspark.sql import functions as F

df2 = df.withColumn("date_str", F.lpad(F.col("date").cast("string"), 8, "0"))

# Now parse based on the apparent MMDDHHmm format
df2 = df2.withColumn("timestamp", F.to_timestamp(F.col("date_str"), "MMddHHmm"))

# Show results
df2.select("date", "timestamp").show(truncate=False)

In [ ]:
spark.sql("""
select delay, origin, destination, 
    Case
        WHEN delay > 360 THEN 'Very Long Delays'
        WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
        WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
        WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
        WHEN delay = 0 THEN 'No Delays'
        ELSE 'Early'
    END as Flight_delays
from temp_table
order by origin, delay DESC
""").show(10)

In [ ]:
df.withColumn(
    "Flight_delays",
    F.when(F.col("delay") > 360, "Very Long Delays")
    .when((F.col("delay") < 360) & (F.col("delay") > 120), "Long Delays")
    .otherwise("Early"),
).show()

There are 2 types of tables:
- **Managed**: Spark manages both the metadata and data itself.
- **Unmanaged**: Spark only manages metadata

Deleting a managed table erases both the metada and data. On the contrary, deleting an unmanaged table only erases metadata

In [ ]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

In [ ]:
# spark.sql("CREATE TABLE managed_temp_table (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

df.write.saveAsTable("managed_table")

In [ ]:
spark.sql("select * from managed_table limit 100").show()

In [ ]:
a = df.select("origin").distinct().toPandas()
destinations = a.origin.to_list()

In [ ]:
df.sampleBy("origin", fractions={i: 0.1 for i in destinations}).show()

Temporary and global temporary views have the following distinction:
- Temporary view can only be used by a single `SparkSession`
- Global temporary view can be used by multiple `SparkSession`

A single application might have more than one `SparkSession`. This is useful when you need to access data with different Hive Metastore configurations

In [ ]:
df_sfo = df.select("date", "delay", "origin", "destination").filter(F.col("origin") == "SFO")
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")

In [ ]:
spark.sql("select count(*) from global_temp.us_origin_airport_SFO_global_tmp_view").show()

In [ ]:
df.schema

schema_ddl = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"

flights = spark.read.format("csv").option("header", "true").schema(schema_ddl).load("data/flights/departuredelays.csv")
flights.show()